In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))

from evaluation import *
from data_loader import *
import random
from datetime import datetime

In [2]:
colors = {
    "DQN": 'darkorange',
    "REINFORCE": 'forestgreen',
    "CMA": 'cyan',
    "ODT": 'blueviolet'
}

In [3]:
starting_exp = 4180
ending_exp = 4188
save_processed_data = True
exp_lists_preformatted = [
    '4000_4017.csv', '4018_4035.csv', '5000_5017.csv', '5018_5035.csv', '6000_6017.csv', '6018_6035.csv', # DQN
    '4036_4053.csv', '4054_4071.csv', '5036_5053.csv', '5054_5071.csv', '6036_6053.csv', '6054_6071.csv', # REINFORCE
    '4072_4089.csv', '4090_4107.csv', '5072_5089.csv', '5090_5107.csv', '6072_6089.csv', '6090_6107.csv', # CMA
    '4108_4125.csv', '4126_4143.csv', '5108_5125.csv', '5126_5143.csv', '6108_6125.csv', '6126_6143.csv', # ODT 1
    '4144_4161.csv', '4162_4179.csv', '5144_5161.csv', '5162_5179.csv', '6144_6161.csv', '6162_6179.csv' # ODT 2
] # Existing formatted datasets
# exp_lists_preformatted = [
#     '4000_4179.csv', '5000_5179.csv', '6000_6179.csv'
# ] # Existing formatted datasets
#exp_lists_preformatted = []

# TODO: REMOVE THIS!!
index = 17 # 13, 15, 16 cause errors
starting_exp = int(exp_lists_preformatted[index].split("_")[0])
ending_exp = int(exp_lists_preformatted[index].split("_")[1].split('.')[0])

exp_lists_preformatted = []

print(f"Running for experiments {starting_exp} to {ending_exp}")

Running for experiments 6090 to 6107


In [4]:
load_agent = True
load_station = True

target_agg_count = 10
target_reward_type = 'greedy'

exp_agent_data = []
exp_station_data = []

experiments = range(starting_exp, ending_exp + 1)

def load_episode_from_csv(filepath, columns=None, target_ep=999, target_agg=9, chunksize=100_000):
    filtered_chunks = []
    for chunk in pd.read_csv(filepath, usecols=columns, chunksize=chunksize):
        sel = (chunk['episode'] == target_ep) & (chunk['aggregation'] == target_agg)
        if sel.any():
            filtered_chunks.append(chunk.loc[sel])
    if filtered_chunks:
        return pd.concat(filtered_chunks, ignore_index=True)
    else:
        return pd.DataFrame(columns=columns or [])


if len(exp_lists_preformatted) == 0:
    for ind, exp_num in enumerate(experiments):
        config_fname = f'./Exp_{exp_num}/config.yaml'
        
        c = load_config_file(config_fname)
        nn_c = c['nn_hyperparameters']
        federated_c = c['federated_learning_settings']
        algo_c = c['algorithm_settings']
        env_c = c['environment_settings']
        
        if federated_c['aggregation_count'] != target_agg_count:
            print(f"Experiment {exp_num} does not have matching aggregation level")
            continue

        rtype = 'communal' if nn_c['average_rewards_when_training'] else 'greedy'
        if rtype != target_reward_type:
            print(f"Experiment {exp_num} does not have matching reward type")
            continue
        
        seed = env_c['seed']
        algorithm_dm = algo_c['algorithm']
        
        d_base = f"../../../../storage_1/metrics/Exp_{exp_num}"
        if not os.path.exists(d_base):
            d_base = f"../metrics/Exp_{exp_num}"
        base_path = f"{d_base}/train/metrics"

        print(f'Loading {base_path}.csv')
        # --- AGENT METRICS ---
        if load_agent:
            cols = [
                'episode', 'timestep', 'done', 'zone', 
                'aggregation', 'reward', 'agent_index',
                'distance', 'average_battery'
            ]
            agent_data = load_episode_from_csv(
                f'{base_path}_agent_metrics.csv',
                columns=cols
            )
            if agent_data.empty:
                print(f"No agent data for ep=10000 in Exp {exp_num}")
            else:
                agent_data['seed'] = seed
                agent_data['exp_num'] = exp_num
                agent_data['algorithm'] = algorithm_dm
                agent_data['season'] = env_c['season']
                exp_agent_data.append(agent_data)

        # --- STATION METRICS ---
        if load_station:
            cols = [
                'episode', 'timestep', 'done', 'zone',
                'aggregation', 'station_index', 'traffic'
            ]
            station_data = load_episode_from_csv(
                f'{base_path}_station_metrics.csv',
                columns=cols
            )
            if station_data.empty:
                print(f"No station data for ep=10000 in Exp {exp_num}")
            else:
                station_data['seed'] = seed
                station_data['exp_num'] = exp_num
                station_data['algorithm'] = algorithm_dm
                station_data['season'] = env_c['season']
                exp_station_data.append(station_data)

Experiment 6090 does not have matching aggregation level
Experiment 6091 does not have matching aggregation level
Experiment 6092 does not have matching aggregation level
Loading ../../../../storage_1/metrics/Exp_6093/train/metrics.csv
Loading ../../../../storage_1/metrics/Exp_6094/train/metrics.csv
Loading ../../../../storage_1/metrics/Exp_6095/train/metrics.csv
Experiment 6096 does not have matching aggregation level
Experiment 6097 does not have matching aggregation level
Experiment 6098 does not have matching aggregation level
Experiment 6099 does not have matching aggregation level
Experiment 6100 does not have matching aggregation level
Experiment 6101 does not have matching aggregation level
Loading ../../../../storage_1/metrics/Exp_6102/train/metrics.csv
Loading ../../../../storage_1/metrics/Exp_6103/train/metrics.csv
Loading ../../../../storage_1/metrics/Exp_6104/train/metrics.csv
Experiment 6105 does not have matching aggregation level
Experiment 6106 does not have matching a

In [5]:
if len(exp_lists_preformatted) == 0:
    # Convert data to DataFrame for easier manipulation
    if load_agent:
        df_agent = pd.concat(exp_agent_data, ignore_index=True)

        cumulative_agent_df = (
            df_agent
            .groupby(
                ['episode', 'zone', 'aggregation', 'agent_index', 'seed', 'exp_num', 'algorithm', 'season']
            )[['distance', 'average_battery', 'reward']]
            .sum()
            .reset_index()
        )

        # Rename the 'reward' column to 'cumulative_reward' for clarity
        cumulative_agent_df.rename(columns={'average_battery': 'battery_used'}, inplace=True)
        cumulative_agent_df.rename(columns={'distance': 'distance_traveled'}, inplace=True)
        
        cumulative_agent_df['episode'] = cumulative_agent_df['aggregation'] * (10000/target_agg_count) + cumulative_agent_df['episode']
    
    if load_station:
        df_station = pd.concat(exp_station_data, ignore_index=True)
    
        print('Cumulating data')    
        cumulative_station_df = (
            df_station
            .groupby(
                ['episode', 'zone', 'aggregation', 'station_index', 'seed', 'exp_num', 'algorithm', 'season']
            )[['traffic']]
            .max()
            .reset_index()
        )

        print('Recalculating episode')    
        # Get recalculated episodes using (aggregation number * episodes per aggregation) + episode number
        cumulative_station_df['episode'] = cumulative_station_df['aggregation'] * (10000/target_agg_count)  + cumulative_station_df['episode']

Cumulating data
Recalculating episode


In [6]:
# Define the file path with starting_exp and ending_exp variables
file_path_for_processed_data = f'../../../../storage_1/metrics/formatted_experiment_data/part_4/'

if len(exp_lists_preformatted) == 0:

    # Get the data for the last episode only
    if load_agent:
        cumulative_agent_df = cumulative_agent_df[cumulative_agent_df['episode'] == cumulative_agent_df['episode'].max()]
    
    if load_station:
        cumulative_station_df = cumulative_station_df[cumulative_station_df['episode'] == cumulative_station_df['episode'].max()]

    
    if save_processed_data:
        if not os.path.exists(file_path_for_processed_data):
            os.makedirs(file_path_for_processed_data)
        
        # Save the DataFrame to a CSV file
        if load_agent:
            cumulative_agent_df.to_csv(f"{file_path_for_processed_data}/agent_{starting_exp}_{ending_exp}.csv", index=False)
            print(f"Data successfully saved to {file_path_for_processed_data}/agent_{starting_exp}_{ending_exp}.csv")

        if load_station:
            cumulative_station_df.to_csv(f"{file_path_for_processed_data}/station_{starting_exp}_{ending_exp}.csv", index=False)
            print(f"Data successfully saved to {file_path_for_processed_data}/station_{starting_exp}_{ending_exp}.csv")

if len(exp_lists_preformatted) > 0:
    if load_agent:
        agent_dataframes = []
        
        # Load and combine datasets
        for existing_set in exp_lists_preformatted:
            loaded_dataset = pd.read_csv(f"{file_path_for_processed_data}/agent_{existing_set}")
            # Combine datasets
            agent_dataframes.append(loaded_dataset)
    
        cumulative_agent_df = pd.concat(agent_dataframes, ignore_index=True)

    if load_station:
        station_dataframes = []
        
        # Load and combine datasets
        for existing_set in exp_lists_preformatted:
            loaded_dataset = pd.read_csv(f"{file_path_for_processed_data}/station_{existing_set}")
            # Combine datasets
            station_dataframes.append(loaded_dataset)
    
        cumulative_station_df = pd.concat(station_dataframes, ignore_index=True)

Data successfully saved to ../../../../storage_1/metrics/formatted_experiment_data/part_4//agent_6090_6107.csv
Data successfully saved to ../../../../storage_1/metrics/formatted_experiment_data/part_4//station_6090_6107.csv


In [7]:
if load_agent:
    algos = cumulative_agent_df['algorithm'].unique()
if load_station:
    algos = cumulative_station_df['algorithm'].unique()

for algo in algos:
    print(f"===== Algorithm: {algo} =====")

    if load_agent:
        agent_d = cumulative_agent_df[cumulative_agent_df['algorithm'] == algo]
        print(f"\tAverage Distance Travelled: {agent_d['distance_traveled'].mean():.2f}")
        print(f"\tDistance Travelled Std Dev: {agent_d['distance_traveled'].std():.2f}")
        print(f"\t--")
        print(f"\tAverage Battery Used (kWh): {agent_d['battery_used'].mean():.2f}")
        print(f"\tBattery Used Std Dev (kWh): {agent_d['battery_used'].std():.2f}")
        print(f"\t--")
        print(f"\tAverage Reward: {agent_d['reward'].mean():.2f}")
        print(f"\tReward Std Dev: {agent_d['reward'].std():.2f}")
        print(f"\t--")
    if load_station:
        station_d = cumulative_station_df[cumulative_station_df['algorithm'] == algo]
        print(f"\tMax Traffic: {station_d['traffic'].max():.2f}")
        #print(f"\tTraffic Std Dev: {station_d['traffic'].std():.2f}")

if load_agent:
    print("\n\n")
    for season in cumulative_agent_df['season'].unique():
        season_data = cumulative_agent_df[cumulative_agent_df['season'] == season]
        
        for algo in season_data['algorithm'].unique():
            algo_data = season_data[season_data['algorithm'] == algo]
    
            avg_energy_used = algo_data['battery_used'].mean()
            std_energy_used = algo_data['battery_used'].std()
    
            print(f"Algorotim: {algo} - Season: {season} \
            - Avg. Energy {avg_energy_used:.2f} - Std. Energy {std_energy_used:.2f}")

===== Algorithm: CMA =====
	Average Distance Travelled: 36.92
	Distance Travelled Std Dev: 1.33
	--
	Average Battery Used (kWh): 22269.95
	Battery Used Std Dev (kWh): 3020.35
	--
	Average Reward: -72.91
	Reward Std Dev: 5.40
	--
	Max Traffic: 24.00



Algorotim: CMA - Season: autumn             - Avg. Energy 21585.36 - Std. Energy 2961.28
Algorotim: CMA - Season: summer             - Avg. Energy 22954.54 - Std. Energy 2923.24
